In [12]:
import pandas as pd
import altair as alt, pandas as pd, numpy as np
alt.data_transformers.disable_max_rows()

# --- Load files (paths from your upload notes) ---
returns = pd.read_csv("20250810_funds_open-end-fund-performance.csv")
profiles = pd.read_csv("20250810_funds_open-end-fund-profile.csv")

# --- Pick a join key that exists in BOTH (prefers stable IDs over names) ---
possible_keys = ["With Intelligence Fund ID", "Fund ID", "Fund Code", "Fund Name"]
join_key = next(k for k in possible_keys if k in returns.columns and k in profiles.columns)

# --- Pick a strategy column from profiles ---
possible_strategy_cols = ["Strategy", "Primary Strategy", "Main Strategy", "Investment Strategy", "Fund Strategy"]
strategy_col = next(c for c in possible_strategy_cols if c in profiles.columns)

# --- Minimal cleanup on the key if it's Fund Name (helps reduce near-dup name mismatches) ---
def _norm_name(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
         .str.replace(r"[’']", "'", regex=True)   # unify apostrophes
    )

if join_key == "Fund Name":
    returns["_key"]  = _norm_name(returns[join_key])
    profiles["_key"] = _norm_name(profiles[join_key])
    key_col = "_key"
else:
    key_col = join_key

# --- Collapse profiles to one row per key with a single strategy (first non-null) ---
prof_strat = (
    profiles[[key_col, strategy_col]]
    .dropna(subset=[key_col])
    .sort_values([key_col])              # deterministic
    .groupby(key_col, as_index=False)
    .agg(**{ "strategy": (strategy_col, "first") })
)

# --- Merge strategy onto monthly returns ---
df = returns.merge(prof_strat, on=key_col, how="left")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7830 entries, 0 to 7829
Data columns (total 84 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Fund ID                                          7830 non-null   int64  
 1   Fund Name                                        7830 non-null   object 
 2   Manager Name                                     7830 non-null   object 
 3   Manager ID                                       7830 non-null   int64  
 4   Fund AuM (m)                                     6301 non-null   float64
 5   Fund AuM Date                                    6301 non-null   object 
 6   Strategy AuM (m)                                 3040 non-null   float64
 7   Strategy AuM Date                                3035 non-null   object 
 8   Last 3 Months                                    7440 non-null   object 
 9   Worst Month                   

In [13]:
# 1) Columns to keep (paste exactly as in your DataFrame)
keep_cols = [
    'Fund Name','Manager Name','Manager ID','Fund AuM (m)',
    'Strategy AuM (m)','Last 3 Months',
    'YTD','Annualized Returns (Since Inception)','2024 Return','2023 Return','2022 Return',
    'Total Return','Positive Months','First Month Reporting Date','3 Year Returns','5 Year Returns',
    'Sharpe Ratio (Since Inception)','Sortino Ratio','Annualized Standard Deviation (Since Inception)',
    'Annualized Upside Deviation (Since Inception)','Annualized Downside Deviation (Since Inception)',
    'Maximum Drawdown','Monthly Return Sep 2024','Monthly Return Oct 2024','Monthly Return Nov 2024',
    'Monthly Return Dec 2024','Monthly Return Jan 2025','Monthly Return Feb 2025',
    'Monthly Return Mar 2025','Monthly Return Apr 2025','Monthly Return May 2025',
    'Monthly Return Jun 2025','Monthly Return Jul 2025','Monthly Return Aug 2025',
    'Monthly Return Sep 2025', 'strategy'
]

# Align with actual columns present (avoids KeyError if any missing)
present_keep_cols = [c for c in keep_cols if c in df.columns]
df = df[present_keep_cols].copy()
df.info()
df.dropna()
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7830 entries, 0 to 7829
Data columns (total 36 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Fund Name                                        7830 non-null   object 
 1   Manager Name                                     7830 non-null   object 
 2   Manager ID                                       7830 non-null   int64  
 3   Fund AuM (m)                                     6301 non-null   float64
 4   Strategy AuM (m)                                 3040 non-null   float64
 5   Last 3 Months                                    7440 non-null   object 
 6   YTD                                              7717 non-null   object 
 7   Annualized Returns (Since Inception)             7707 non-null   object 
 8   2024 Return                                      7732 non-null   object 
 9   2023 Return                   

In [14]:
import pandas as pd
import altair as alt

# --- detect a strategy column ---
strategy_col = next((c for c in ["strategy","Strategy","Primary Strategy","Fund Strategy","Main Strategy"]
                     if c in df.columns), None)
assert strategy_col is not None, "No strategy column found."

# --- tidy ---
rp = df.rename(columns={
    "Manager Name":"manager",
    "Sharpe Ratio (Since Inception)":"sharpe",
    "Sortino Ratio":"sortino",
    "Annualized Standard Deviation (Since Inception)":"stdev",
    strategy_col: "strategy",
})
rp["stdev"] = pd.to_numeric(rp["stdev"].astype(str).str.replace("%","", regex=False),
                            errors="coerce")/100

# --- aggregate to one row per manager ---
def _mode_or_first(s: pd.Series):
    m = s.mode()
    return m.iloc[0] if not m.empty else s.dropna().iloc[0] if s.dropna().size else None

mgr = (
    rp.dropna(subset=["manager","sharpe","stdev"])
      .groupby("manager", as_index=False)
      .agg(sharpe=("sharpe","mean"),
           sortino=("sortino","mean"),
           stdev=("stdev","mean"),
           strategy=("strategy", _mode_or_first),
           n_funds=("manager","size"))
)

# --- chart: each point = manager, color = strategy ---
select_strat = alt.selection_multi(fields=["strategy"], bind="legend")  # click legend to filter/highlight

chart = (
  alt.Chart(mgr)
    .mark_circle(size=110, opacity=0.9, stroke='black', strokeWidth=0.25)
    .encode(
        x=alt.X("stdev:Q",  title="Avg Risk (Stdev)", axis=alt.Axis(format="%")),
        y=alt.Y("sharpe:Q", title="Avg Sharpe"),
        color=alt.Color("strategy:N", title="Strategy", legend=alt.Legend(symbolLimit=30)),
        tooltip=[
            alt.Tooltip("manager:N",  title="Manager"),
            alt.Tooltip("strategy:N", title="Strategy"),
            alt.Tooltip("sharpe:Q",   format=".2f"),
            alt.Tooltip("sortino:Q",  format=".2f"),
            alt.Tooltip("stdev:Q",    format=".1%"),
            alt.Tooltip("n_funds:Q",  title="# Funds")
        ],
        opacity=alt.condition(select_strat, alt.value(1), alt.value(0.25))
    )
    .properties(width=780, height=440, title="Managers: Risk vs. Sharpe (color = Strategy)")
    .add_selection(select_strat)
    .interactive()
)

chart

/var/folders/3z/qc34j53n3mv8yp0p4bksgypm0000gn/T/ipykernel_84336/1768090761.py:36: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  select_strat = alt.selection_multi(fields=["strategy"], bind="legend")  # click legend to filter/highlight
/var/folders/3z/qc34j53n3mv8yp0p4bksgypm0000gn/T/ipykernel_84336/1768090761.py:56: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(select_strat)


alt.Chart(...)

In [15]:
rr = df[[
    "Fund Name","Fund AuM (m)","Sharpe Ratio (Since Inception)",
    "Annualized Returns (Since Inception)","Annualized Standard Deviation (Since Inception)"
]].rename(columns={
    "Fund AuM (m)":"aum", "Sharpe Ratio (Since Inception)":"sharpe",
    "Annualized Returns (Since Inception)":"ret", "Annualized Standard Deviation (Since Inception)":"stdev"
})

# quick percent coercion (works for numbers or '12.3%')
for c in ["ret","stdev"]:
    rr[c] = pd.to_numeric(rr[c].astype(str).str.replace("%","", regex=False), errors="coerce")/100

rr = rr.dropna(subset=["ret","stdev","sharpe","aum"])
xmax = rr["stdev"].quantile(0.99); ymin, ymax = rr["ret"].quantile([0.01,0.99])
brush = alt.selection_interval()   # drag to zoom/select

chart1 = (
  alt.Chart(rr.query("stdev<=@xmax and @ymin<=ret<=@ymax"))
    .mark_circle(opacity=0.75)
    .encode(
        x=alt.X("stdev:Q", title="Risk (Ann. Stdev)", axis=alt.Axis(format="%")),
        y=alt.Y("ret:Q",   title="Return (Ann.)",     axis=alt.Axis(format="%")),
        size=alt.Size("aum:Q", title="AuM (m)", scale=alt.Scale(range=[20,350])),
        color=alt.Color("sharpe:Q", title="Sharpe", scale=alt.Scale(scheme="viridis")),
        tooltip=["Fund Name","sharpe:Q","aum:Q","ret:Q","stdev:Q"]
    )
    .properties(width=700, height=430, title="Risk–Return (AuM size, Sharpe color)")
    .add_selection(brush)
    .interactive()
)
chart1

/var/folders/3z/qc34j53n3mv8yp0p4bksgypm0000gn/T/ipykernel_84336/322372388.py:28: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(brush)


alt.Chart(...)